# A1: Data Curation
Karl Stavem  
ID:  1978397

---
> TL;DR: The goal of this assignment is to construct, analyze, and publish a dataset of monthly traffic on English Wikipedia from January 1, 2008 through August 30, 2020.
---

### Step 1: Gathering the data

In [1]:
# import libraries
import os
import requests
import json
import pandas as pd
import numpy as np

Since there are two separate APIs, define the endpoints.

In [2]:
# set the API endpoint

# from January 2008 to July 2016
endpoint_legacy = 'https://wikimedia.org/api/rest_v1/metrics/legacy/pagecounts/aggregate/{project}/{access-site}/{granularity}/{start}/{end}'

# from May 1st, 2015
endpoint_new = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/aggregate/{project}/{access}/{agent}/{granularity}/{start}/{end}'

First, make sure header information is set for each API call.

In [3]:
# set headers for API calls
headers = {
    'User-Agent': 'https://github.com/stavem',
    'From': 'kstavem@uw.edu'
}

Because the parameters are similiar in each API but the syntax is slightly different between them, create a method to quickly switch between the two.   For this exercise, most of our parameters remain constant so we only need to change the version and access site.  We could also leave the start and end date constant if we like to catch all possible data in each API.

In [4]:
def assign_parameters(is_legacy_version, access, project = "en.wikipedia.org", agent = "user", granularity = "monthly", 
                      start = '2008010100', end =  '2020090100'):
    """
    Input:  Access and is_legacy_version
            For legacy API set is_legacy_version = 1
            For legacy API:  Valid access inputs in ['dektop-site', 'mobile-site']
            For non-legacy API:  Valid access in ['desktop', 'mobile-web', 'mobile-app']
        
    Output:  Parameters for use in API query
    """
    
    if is_legacy_version:
                      params = {"project" : project,
                                "access-site" : access,
                                "granularity" : granularity,
                                "start" : start,
                                "end" : end
                               }

    else:
                      params = {"project" : project,
                                "access" :access,
                                "agent" : agent,
                                "granularity" : granularity,
                                "start" : start,
                                "end" : end
                               }
    return params

Create a method to pass these parameters into a wikipedia API call and return the response.

In [5]:
def api_call(endpoint,parameters):
    """
    Conduct API calls
    Inputs:  Wikipedia API Endpoint, Parameters for API call
    Output:  Response in JSON format
    """
    
    # try to call the api
    try:
        call = requests.get(endpoint.format(**parameters), headers=headers)
        call.raise_for_status()
        
    # Alert user of any errors
    except requests.exceptions.HTTPError as error:
        print(error)
    
    response = call.json()
    return response

In [6]:
# define the 5 sites we'll be looking at
new_sites = ['desktop', 'mobile-web', 'mobile-app']
legacy_sites = ['desktop-site', 'mobile-site']

In [7]:
# set directory to work with all saved files
directory_path = '/home/jovyan/data-512/data-512-a1/'

# store list of saved files for later
files_list = []

Collect all 3 data sources from the new API.   Save the results as JSON files.  Store the file names in a list for access later.

In [10]:
for site in new_sites:
    
    # call the API
    params = assign_parameters(is_legacy_version = 0, access = site, start = '2015070100', end = '2020080100')
    data = api_call(endpoint_new, params)
    
    # name the file, store location in a list
    filename = 'pageviews_{}_201507-202008.json'.format(site)
    files_list.append(directory_path + filename)
    
    # write to file
    with open(directory_path + filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    
    

Collect all 3 data sources from the legacy API.   Save the results as JSON files.  Store the file names in a list for access later.

In [11]:
for site in legacy_sites:
    
    # call the API
    params = assign_parameters(is_legacy_version = 1, access = site, start = '2007120100', end = '2016080100')
    data = api_call(endpoint_legacy, params)
    
    # name the file, store location in a list
    filename = 'legacy_pagecounts_{}_200712-201607.json'.format(site)
    files_list.append(directory_path + filename)
    
    # write to file
    with open(directory_path + filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
files_list


### Step 2: Processing the data

* For data collected from the Pageviews API, combine the monthly values for mobile-app and mobile-web to create a total mobile traffic count for each month.

First, iterate through all the files that have been collected and combine them into one big JSON file.

In [113]:
# combine all json files into one json file
combined_json_file=[]

for file in files_list:
    with open(file) as f:
        combined_json_file.append(json.load(f))

Then, convert that JSON file into a data frame to make it easier to manipulate.

In [114]:
# convert JSON file into pandas dataframe
df = pd.DataFrame()
for i in range(len(combined_json_file)):
    df=df.append(pd.json_normalize(combined_json_file[i]['items']))
    
# check results
display(df)
                                   

,project,access,agent,granularity,timestamp,views,access-site,count
0,en.wikipedia,desktop,user,monthly,2015070100,4.376667e+09,NaN,NaN
1,en.wikipedia,desktop,user,monthly,2015080100,4.332482e+09,NaN,NaN
2,en.wikipedia,desktop,user,monthly,2015090100,4.485492e+09,NaN,NaN
3,en.wikipedia,desktop,user,monthly,2015100100,4.477533e+09,NaN,NaN
4,en.wikipedia,desktop,user,monthly,2015110100,4.287720e+09,NaN,NaN
...,...,...,...,...,...,...,...,...
17,en.wikipedia,NaN,NaN,monthly,2016030100,NaN,mobile-site,3.419854e+09
18,en.wikipedia,NaN,NaN,monthly,2016040100,NaN,mobile-site,3.301385e+09
19,en.wikipedia,NaN,NaN,monthly,2016050100,NaN,mobile-site,3.418436e+09
20,en.wikipedia,NaN,NaN,monthly,2016060100,NaN,mobile-site,3.372618e+09


In [115]:
# keep track of the api that was used
df['API'] = np.where(df['access'].isna(), 'pagecount', 'pageviews')

In [116]:
# drop the project, granularity columns
df.drop(['project', 'granularity', 'agent'], axis=1, inplace = True)


In [117]:
# combine all the newest mobile traffic into one category
df.loc[df['access'].isin(['mobile-app', 'mobile-web']),'access'] = 'pageview_mobile_views'

# change the desktop value
df.loc[df['access'].isin(['desktop']),'access'] = 'pageview_desktop_views'

# combine all the newest mobile traffic into one category
df.loc[df['access-site'].isin(['mobile-site']),'access-site'] = 'pagecount_mobile_views'

# change the value to desktop to desktop-site
df.loc[df['access-site'].isin(['desktop-site']),'access-site'] = 'pagecount_desktop_views'


In [118]:
# create a new source column to track
df['source'] = np.where(df["access"].isnull(), df["access-site"], df["access"] )

#drop old columns
df.drop(['access', 'access-site'], axis=1, inplace = True)

In [119]:
# extract the year and the month from the timestamp column
df['year'] = df['timestamp'].str[:4]
df['month'] = df['timestamp'].str[4:6]

# drop the timestamp column as it is unnecessary
df.drop(['timestamp'], axis=1, inplace = True)


In [120]:
# get rid of NA in total columns
df['views'] = df['views'].fillna(0)
df['count'] = df['count'].fillna(0)

# add totals
df['views'] = df['views']+df['count'].fillna(0)

# drop the count column
df.drop(['count'], axis=1, inplace = True)

Combine all data into a single CSV file with the following headers:

* year / YYYY
* month / MM
* pagecount_all_views / num_views
* pagecount_desktop_views / num_views
* pagecount_mobile_views / num_views
* pageview_all_views / num_views
* pageview_desktop_views / num_views
* pageview_mobile_views / num_views


In [122]:
df_pivoted = pd.pivot_table(df, values='views', index=['year', 'month'],
                    columns=['source'], aggfunc=np.sum, fill_value=0)

# Formatting.
df_pivoted.reset_index(inplace=True)
df_pivoted.columns.name = None

In [132]:
df_pivoted['pagecount_all_views'] = df_pivoted['pagecount_desktop_views'] + df_pivoted['pagecount_mobile_views']
df_pivoted['pageview_all_views'] = df_pivoted['pageview_desktop_views'] + df_pivoted['pageview_mobile_views']

In [133]:
df_pivoted

,year,month,pagecount_desktop_views,pagecount_mobile_views,pageview_desktop_views,pageview_mobile_views,pagecount_all_views,pageview_all_views
0,2007,12,5996663048,0,0,0,5996663048,0
1,2008,01,9861805140,0,0,0,9861805140,0
2,2008,02,9636787526,0,0,0,9636787526,0
3,2008,03,9910811618,0,0,0,9910811618,0
4,2008,04,10318324366,0,0,0,10318324366,0
...,...,...,...,...,...,...,...,...
147,2020,03,0,0,6824996548,9372541110,0,16197537658
148,2020,04,0,0,7596745226,11011483882,0,18608229108
149,2020,05,0,0,6156187230,10463400190,0,16619587420
150,2020,06,0,0,5442657114,9147950512,0,14590607626


For all months with 0 pageviews for a given access method (e.g. desktop-site, mobile-app), that value for that (column, month) should be listed as 0. So for example all values of pagecount_mobile_views for months before October 2014 should be 0, because mobile traffic data is not available before that month.

In [39]:
df

,access,views,access-site,count,API,year,month,source
0,desktop,4.376667e+09,NaN,NaN,pageviews,2015,07,Mismatch
1,desktop,4.332482e+09,NaN,NaN,pageviews,2015,08,Mismatch
2,desktop,4.485492e+09,NaN,NaN,pageviews,2015,09,Mismatch
3,desktop,4.477533e+09,NaN,NaN,pageviews,2015,10,Mismatch
4,desktop,4.287720e+09,NaN,NaN,pageviews,2015,11,Mismatch
...,...,...,...,...,...,...,...,...
17,NaN,NaN,mobile-site,3.419854e+09,pagecount,2016,03,Mismatch
18,NaN,NaN,mobile-site,3.301385e+09,pagecount,2016,04,Mismatch
19,NaN,NaN,mobile-site,3.418436e+09,pagecount,2016,05,Mismatch
20,NaN,NaN,mobile-site,3.372618e+09,pagecount,2016,06,Mismatch


The final data file should be named:

en-wikipedia_traffic_200712-202008.csv

In [ ]:
# write file
df.to_csv('en-wikipedia_traffic_200712-202008.csv', index=False)

### Step 3: Analyze the data

### Step 4: Document your process thoroughly

---
### Step 5: Submit the assignment

The github link for this assignment can be found here:   <a href="https://github.com/stavem/data-512/tree/main/data-512-a1">here</a>.

The github link for this assignment can be found here:   <a href="https://github.com/stavem/data-512/tree/main/data-512-a1">here</a>.